<a href="https://colab.research.google.com/github/sagar9926/MTech_Atificial_Intelligence/blob/main/AI_1/Assignment_1/AI_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from random import randint
import sys
import time
import math
import itertools
import copy 
import numpy as np
import random
np.random.seed(42)
!git clone https://github.com/sagar9926/MTech_Atificial_Intelligence.git


Cloning into 'MTech_Atificial_Intelligence'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 302 (delta 116), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (302/302), 2.55 MiB | 3.95 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [5]:
cd /content/MTech_Atificial_Intelligence/AI_1/Assignment_1


/content/MTech_Atificial_Intelligence/AI_1/Assignment_1


In [6]:
input_file = 'input.txt'
f = open(input_file)


In [7]:
# each session has k papers
k=int(f.readline()) 

# schedule have p parallel sessions
p=int(f.readline()) 

# total of t timeslots
t=int(f.readline())

c=int(f.readline())

# total number of paper
tnp=p*k*t 

dim=k*p*t
dis=[]  #Distance array

sim=[]  #Similarity array


schedule = [[[None for _ in range(k)] for _ in range(p)] for _ in range(t)] #This denotes the state of my search problem 

In [8]:
for i in range(dim):
  m=[]
  l=f.readline().strip().split(' ')
  for i in range(len(l)):
    l[i]=float(l[i])
    m+=[round(1.0-float(l[i]),2)]
  dis+=[l]
  sim+=[m]

In [9]:
dis

[[0.0, 0.4, 0.8, 1.0],
 [0.4, 0.0, 0.6, 0.7],
 [0.8, 0.6, 0.0, 0.3],
 [1.0, 0.7, 0.3, 0.0]]

In [10]:
sim

[[1.0, 0.6, 0.2, 0.0],
 [0.6, 1.0, 0.4, 0.3],
 [0.2, 0.4, 1.0, 0.7],
 [0.0, 0.3, 0.7, 1.0]]

In [11]:
def goodness_function(schedule,dis,sim):
  goodness = {}

  for time_slot_index in range(len(schedule)):
    goodness[time_slot_index] = 0 

    # fetch the schedule for one time slot
    single_time_slot_sessions = schedule[time_slot_index]

    # Calculate similarity score and update goodness
    for index in range(len(single_time_slot_sessions)):
      session = single_time_slot_sessions[index]
      for session_i in session:
        for session_j in session:
          if(session_i != session_j):
            goodness[time_slot_index] += sim[session_i][session_j] / 2

    # Calculate dissimilarity score and update goodness
    for session_a in single_time_slot_sessions:
      for session_b in single_time_slot_sessions:
        if session_a != session_b:
          for session_i in session_a:
            for session_j in session_b:
              goodness[time_slot_index] += dis[session_i][session_j] / 2

  return goodness

In [12]:
def generate_neighbour (current_schedule_state):
  papers = sum(list(itertools.chain(*current_schedule_state)),[])
  global t
  global p
  global k
  next_schedule_state = [[[None for _ in range(k)] for _ in range(p)] for _ in range(t)]

  for slot in range(t):
    for session in range(p):
      for paper in range(k):
        temp = random.sample(papers,k = 1)[0]
        next_schedule_state[slot][session][paper] = temp
        papers.remove(temp)
  return next_schedule_state

In [13]:
np.random.randint(k),np.random.randint(p),np.random.randint(t)

(0, 1, 0)

In [14]:
def generate_neighbour2 (current_schedule_state):
  papers = sum(list(itertools.chain(*current_schedule_state)),[])
  global t
  global p
  global k
  next_schedule_state = copy.deepcopy(current_schedule_state)
  slot1,session1,paper1  = np.random.randint(t),np.random.randint(p),np.random.randint(k)
  slot2,session2,paper2  = np.random.randint(t),np.random.randint(p),np.random.randint(k)

  temp = next_schedule_state[slot2][session2][paper2]
  next_schedule_state[slot2][session2][paper2] = next_schedule_state[slot1][session1][paper1]
  next_schedule_state[slot1][session1][paper1] = temp
  
  return next_schedule_state

In [15]:
def simulated_annealing(initial_schedule_state):
    """Peforms simulated annealing to find a solution"""

    # Exploration to find best initial state
    exploration = 100
    update_exploration = 0.001

    while exploration > 0.1:
      exploration = exploration - update_exploration
      neighbor = generate_neighbour(initial_schedule_state)

        # Check if neighbor is best so far
      cost_diff_1 = sum([next_cost - current_cost for current_cost , next_cost  in  zip(goodness_function(initial_schedule_state,dis,sim).values() , goodness_function(neighbor,dis,sim).values())])

        # if the new solution is better, accept it
      if cost_diff_1 > 0:
        best_initial_state = neighbor
        initial_schedule_state = neighbor

    print('best_initial_state',best_initial_state,"and it's cost is",goodness_function(best_initial_state,dis,sim))
    # Exploration + Exploitation
    initial_temp = 180
    final_temp = .1
    alpha = 0.001
    
    current_temp = initial_temp

    # Start by initializing the current state with the initial state
    current_schedule_state = best_initial_state
    solution = best_initial_state

    while current_temp > final_temp:
        neighbor = generate_neighbour2(current_schedule_state)

        # Check if neighbor is best so far
        cost_diff = sum([next_cost - current_cost for current_cost , next_cost  in  zip(goodness_function(current_schedule_state,dis,sim).values() , goodness_function(neighbor,dis,sim).values())])
        # if the new solution is better, accept it
        if cost_diff > 0:
            solution = neighbor
            current_schedule_state = neighbor
            last_best_state = neighbor
        # if the new solution is not better, accept it with a probability of e^(-cost/temp)
        else:
            if random.uniform(0, 1) < math.exp(cost_diff / current_temp):
                solution = neighbor
                current_schedule_state = neighbor
        # decrement the temperature
        current_temp -= alpha

    return last_best_state , solution


In [16]:
schedule

[[[None, None], [None, None]]]

In [17]:
initial_state = [[[1, 3], [2, 0]]] # Selected randomly
initial_state = generate_neighbour(initial_state)
print("Initial state :",initial_state )
last_best_state,result  = simulated_annealing(initial_state)
print("Cost of Result : ",result," is ",goodness_function(result,dis,sim))

Initial state : [[[0, 3], [2, 1]]]
best_initial_state [[[1, 0], [3, 2]]] and it's cost is {0: 4.4}
Cost of Result :  [[[2, 3], [1, 0]]]  is  {0: 4.4}
